# Clustering Exercises - Acquire

### Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.


**acquire & summarize**

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

    - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    
    - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
    
    - Only include properties that include a latitude and longitude value.


In [8]:
#basic imports
import pandas as pd
import numpy as np
import env

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Clear pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# Exploring
import scipy.stats as stats

# Handle large numbers w/o using scientific notation
pd.options.display.float_format = '{:.3f}'.format

In [23]:
import env

# connection function for accessing mysql 
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def acquire(df):
    '''
    This function connects to Codeup's SQL Server using given parameters in the user's
    env file.  It then uses a SQL query to acquire all data from Zillow's database that has a transaction data in 2017 and
    has longitude and latitude information on the property.
    
    It returns all the data in a single dataframe called df.
    '''
    
    def get_connection(db, user=env.user, host=env.host, password=env.password):
         return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    query = '''
            SELECT *
            FROM properties_2017 
                left join predictions_2017 using (parcelid)
                left join airconditioningtype USING (airconditioningtypeid)
                left join architecturalstyletype USING (architecturalstyletypeid)
                left join buildingclasstype USING (buildingclasstypeid)
                left join heatingorsystemtype USING (heatingorsystemtypeid)
                left join propertylandusetype USING (propertylandusetypeid)
                left join storytype USING (storytypeid)
                left join typeconstructiontype USING (typeconstructiontypeid)
            WHERE ((longitude is not null) and 
                   (latitude is not null) and 
                   (transactiondate like '2017%'));
            '''

    df = pd.read_sql(query, get_connection('zillow'))
    
    return df

df = acquire('zillow')
df.head(2)

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,nan,nan,261.000,nan,nan,nan,nan,14297519,1727539,nan,...,0,0.026,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,nan,nan,261.000,nan,nan,nan,nan,17052889,1387261,nan,...,1,0.056,2017-01-01,None,None,None,None,Single Family Residential,None,None


In [10]:
# # google sheet: sheet_url = 'https://docs.google.com/spreadsheets/d/14L32EfCmr2asv85i08fla6jf1KakJVcLYaJMkXQ4_p0/edit#gid=0'    

# #Note: Data has been filtered/changed a bit from orginal form to demonstrate null and outlier handling.

# sheet_url = 'https://docs.google.com/spreadsheets/d/1Zgcrj5n4G75rwyMP1pvdCXw8kLYcqsPFWYg4mr4seKM/edit#gid=0'

# csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

# df = pd.read_csv(csv_export_url)
    
    

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [6]:
df.describe()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id,logerror
count,223.000,50.000,77579.000,49571.000,15.000,207.000,25007.000,77579.000,77579.000,50.000,...,172.000,77464.000,77578.000,77579.000,77577.000,77574.000,2900.000,77332.000,77579.000,77579.000
mean,6.040,7.000,261.824,3.922,3.933,7.386,1.812,13008278.060,1495391.586,679.720,...,1.000,189281.085,490150.589,2016.000,301151.533,5995.962,14.088,60496673885751.219,38806.224,0.017
std,0.556,0.000,5.142,3.595,0.258,2.728,2.966,3519398.646,860968.646,689.704,...,0.000,230410.683,653797.860,0.000,492724.916,7628.860,2.181,1533339296950.520,22403.467,0.171
min,4.000,7.000,31.000,1.000,3.000,2.000,1.000,10711855.000,349.000,38.000,...,1.000,44.000,1000.000,2016.000,161.000,19.920,3.000,60371011101000.000,0.000,-4.655
25%,6.000,7.000,261.000,2.000,4.000,7.000,1.000,11538198.000,752142.000,273.000,...,1.000,84179.250,206898.500,2016.000,85293.000,2712.630,14.000,60373109004005.000,19404.500,-0.024
50%,6.000,7.000,261.000,2.000,4.000,7.000,1.000,12530543.000,1498227.000,515.000,...,1.000,136404.500,358878.500,2016.000,203188.000,4448.265,15.000,60376032001510.500,38804.000,0.007
75%,6.000,7.000,266.000,7.000,4.000,7.000,1.000,14211357.500,2240879.000,796.500,...,1.000,218734.000,569000.000,2016.000,366753.000,6926.917,15.000,60590423251007.000,58207.500,0.039
max,13.000,7.000,275.000,24.000,4.000,21.000,13.000,167689317.000,2982274.000,3560.000,...,1.000,11421790.000,49061236.000,2016.000,48952198.000,586639.300,99.000,483030105084015.000,77612.000,5.263


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        223 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77579 non-null  float64
 3   heatingorsystemtypeid         49571 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      207 non-null    float64
 6   airconditioningtypeid         25007 non-null  float64
 7   parcelid                      77579 non-null  int64  
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

In [8]:
df.columns

Index(['typeconstructiontypeid', 'storytypeid', 'propertylandusetypeid',
       'heatingorsystemtypeid', 'buildingclasstypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'parcelid', 'id',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17',

In [9]:
# How many nulls in each column?
df.isnull().sum()

typeconstructiontypeid    77356
storytypeid               77529
propertylandusetypeid         0
heatingorsystemtypeid     28008
buildingclasstypeid       77564
                          ...  
buildingclassdesc         77564
heatingorsystemdesc       28008
propertylandusedesc           0
storydesc                 77529
typeconstructiondesc      77356
Length: 69, dtype: int64

In [10]:
# % values missing in each column

df.isnull().sum()/len(df)*100

typeconstructiontypeid   99.713
storytypeid              99.936
propertylandusetypeid     0.000
heatingorsystemtypeid    36.103
buildingclasstypeid      99.981
                          ...  
buildingclassdesc        99.981
heatingorsystemdesc      36.103
propertylandusedesc       0.000
storydesc                99.936
typeconstructiondesc     99.713
Length: 69, dtype: float64

In [11]:
df.shape

(77579, 69)

In [12]:
#How many nulls in each row?

df.isnull().sum(axis=1).value_counts()

33    14819
34    13357
32    12606
31     9911
36     5793
29     5285
35     5163
37     3630
30     3465
38     1932
28      457
27      316
39      285
40      230
44       79
26       66
45       50
41       29
43       29
42       27
25       24
24       13
46        5
48        3
47        3
23        2
dtype: int64

In [13]:
df.value_counts('parcelid')

parcelid
12612211    3
11991059    3
10857130    3
12136147    2
11705026    2
           ..
13889477    1
13889479    1
13889481    1
13889489    1
10711855    1
Length: 77381, dtype: int64

In [14]:

df.nunique(axis = 0)

typeconstructiontypeid     4
storytypeid                1
propertylandusetypeid     13
heatingorsystemtypeid     10
buildingclasstypeid        2
                          ..
buildingclassdesc          2
heatingorsystemdesc       10
propertylandusedesc       13
storydesc                  1
typeconstructiondesc       4
Length: 69, dtype: int64

In [15]:
#drop columns with > 50% nulls
threshold = int(0.5 * len(df.index)) # Require that many non-NA values.
df.dropna(axis = 1, thresh = threshold, inplace = True)
df.head()

,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,261.000,nan,14297519,1727539,3.500,4.000,nan,3.500,3100.000,3100.000,...,1023282.000,2016.000,537569.000,11013.720,60590630072012.000,0,0.026,2017-01-01,None,Single Family Residential
1,261.000,nan,17052889,1387261,1.000,2.000,nan,1.000,1465.000,1465.000,...,464000.000,2016.000,376000.000,5672.480,61110010023006.000,1,0.056,2017-01-01,None,Single Family Residential
2,261.000,nan,14186244,11677,2.000,3.000,nan,2.000,1243.000,1243.000,...,564778.000,2016.000,479489.000,6488.300,60590218022012.000,2,0.005,2017-01-01,None,Single Family Residential
3,261.000,2.000,12177905,2288172,3.000,4.000,8.000,3.000,2376.000,2376.000,...,145143.000,2016.000,36225.000,1777.510,60373001001006.000,3,-0.103,2017-01-01,Central,Single Family Residential
4,266.000,2.000,10887214,1970746,3.000,3.000,8.000,3.000,1312.000,1312.000,...,119407.000,2016.000,45726.000,1533.890,60371236012000.000,4,0.007,2017-01-01,Central,Condominium


In [16]:
#drop rows with > 50% nulls
threshold = int(0.5 * len(df.columns)) # Require that many non-NA values.
df.dropna(axis = 0, thresh = threshold, inplace = True)

df

,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,261.000,nan,14297519,1727539,3.500,4.000,nan,3.500,3100.000,3100.000,...,1023282.000,2016.000,537569.000,11013.720,60590630072012.000,0,0.026,2017-01-01,None,Single Family Residential
1,261.000,nan,17052889,1387261,1.000,2.000,nan,1.000,1465.000,1465.000,...,464000.000,2016.000,376000.000,5672.480,61110010023006.000,1,0.056,2017-01-01,None,Single Family Residential
2,261.000,nan,14186244,11677,2.000,3.000,nan,2.000,1243.000,1243.000,...,564778.000,2016.000,479489.000,6488.300,60590218022012.000,2,0.005,2017-01-01,None,Single Family Residential
3,261.000,2.000,12177905,2288172,3.000,4.000,8.000,3.000,2376.000,2376.000,...,145143.000,2016.000,36225.000,1777.510,60373001001006.000,3,-0.103,2017-01-01,Central,Single Family Residential
4,266.000,2.000,10887214,1970746,3.000,3.000,8.000,3.000,1312.000,1312.000,...,119407.000,2016.000,45726.000,1533.890,60371236012000.000,4,0.007,2017-01-01,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,266.000,2.000,10833991,2864704,3.000,3.000,8.000,3.000,1741.000,1741.000,...,379000.000,2016.000,114000.000,4685.340,60371319001006.000,77608,-0.002,2017-09-20,Central,Condominium
77575,261.000,2.000,11000655,673515,2.000,2.000,6.000,2.000,1286.000,1286.000,...,354621.000,2016.000,283704.000,4478.430,60371014003002.000,77609,0.021,2017-09-20,Central,Single Family Residential
77576,261.000,nan,17239384,2968375,2.000,4.000,nan,2.000,1612.000,1612.000,...,67205.000,2016.000,16522.000,1107.480,61110084022016.000,77610,0.013,2017-09-21,None,Single Family Residential
77577,261.000,2.000,12773139,1843709,1.000,3.000,4.000,1.000,1032.000,1032.000,...,49546.000,2016.000,16749.000,876.430,60374338022005.000,77611,0.037,2017-09-21,Central,Single Family Residential


3.  Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an attribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

Example: 

In [17]:
#def count_nulls():
#df.set_index('parcelid', inplace = True)


In [13]:
df

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,nan,nan,261.000,nan,nan,nan,nan,14297519,1727539,nan,...,0,0.026,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,nan,nan,261.000,nan,nan,nan,nan,17052889,1387261,nan,...,1,0.056,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,nan,nan,261.000,nan,nan,nan,nan,14186244,11677,nan,...,2,0.005,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,nan,nan,261.000,2.000,nan,nan,nan,12177905,2288172,nan,...,3,-0.103,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,nan,nan,266.000,2.000,nan,nan,1.000,10887214,1970746,nan,...,4,0.007,2017-01-01,Central,None,None,Central,Condominium,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,nan,nan,266.000,2.000,nan,nan,1.000,10833991,2864704,nan,...,77608,-0.002,2017-09-20,Central,None,None,Central,Condominium,None,None
77575,nan,nan,261.000,2.000,nan,nan,nan,11000655,673515,nan,...,77609,0.021,2017-09-20,None,None,None,Central,Single Family Residential,None,None
77576,nan,nan,261.000,nan,nan,nan,nan,17239384,2968375,nan,...,77610,0.013,2017-09-21,None,None,None,None,Single Family Residential,None,None
77577,nan,nan,261.000,2.000,nan,nan,1.000,12773139,1843709,nan,...,77611,0.037,2017-09-21,Central,None,None,Central,Single Family Residential,None,None


In [21]:
rows = df.shape[0]
nulls = pd.DataFrame(df.isnull().sum())
nulls.columns = ['null_count']
nulls['null_percentage'] = (nulls / rows) * 100
nulls

,null_count,null_percentage
typeconstructiontypeid,77356,99.713
storytypeid,77529,99.936
propertylandusetypeid,0,0.000
heatingorsystemtypeid,28008,36.103
buildingclasstypeid,77564,99.981
...,...,...
heatingorsystemdesc,28008,36.103
propertylandusedesc,0,0.000
storydesc,77529,99.936
typeconstructiondesc,77356,99.713


In [15]:
df['Missing values'] = df.isnull().sum(axis =0).value_counts()
df

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,Missing values
0,nan,nan,261.000,nan,nan,nan,nan,14297519,1727539,nan,...,0.026,2017-01-01,None,None,None,None,Single Family Residential,None,None,17.000
1,nan,nan,261.000,nan,nan,nan,nan,17052889,1387261,nan,...,0.056,2017-01-01,None,None,None,None,Single Family Residential,None,None,1.000
2,nan,nan,261.000,nan,nan,nan,nan,14186244,11677,nan,...,0.005,2017-01-01,None,None,None,None,Single Family Residential,None,None,1.000
3,nan,nan,261.000,2.000,nan,nan,nan,12177905,2288172,nan,...,-0.103,2017-01-01,None,None,None,Central,Single Family Residential,None,None,nan
4,nan,nan,266.000,2.000,nan,nan,1.000,10887214,1970746,nan,...,0.007,2017-01-01,Central,None,None,Central,Condominium,None,None,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,nan,nan,266.000,2.000,nan,nan,1.000,10833991,2864704,nan,...,-0.002,2017-09-20,Central,None,None,Central,Condominium,None,None,nan
77575,nan,nan,261.000,2.000,nan,nan,nan,11000655,673515,nan,...,0.021,2017-09-20,None,None,None,Central,Single Family Residential,None,None,nan
77576,nan,nan,261.000,nan,nan,nan,nan,17239384,2968375,nan,...,0.013,2017-09-21,None,None,None,None,Single Family Residential,None,None,nan
77577,nan,nan,261.000,2.000,nan,nan,1.000,12773139,1843709,nan,...,0.037,2017-09-21,Central,None,None,Central,Single Family Residential,None,None,nan


,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc,Missing values
0,261.000,nan,14297519,1727539,3.500,4.000,nan,3.500,3100.000,3100.000,...,2016.000,537569.000,11013.720,60590630072012.000,0,0.026,2017-01-01,None,Single Family Residential,17.000
1,261.000,nan,17052889,1387261,1.000,2.000,nan,1.000,1465.000,1465.000,...,2016.000,376000.000,5672.480,61110010023006.000,1,0.056,2017-01-01,None,Single Family Residential,1.000
2,261.000,nan,14186244,11677,2.000,3.000,nan,2.000,1243.000,1243.000,...,2016.000,479489.000,6488.300,60590218022012.000,2,0.005,2017-01-01,None,Single Family Residential,1.000
3,261.000,2.000,12177905,2288172,3.000,4.000,8.000,3.000,2376.000,2376.000,...,2016.000,36225.000,1777.510,60373001001006.000,3,-0.103,2017-01-01,Central,Single Family Residential,nan
4,266.000,2.000,10887214,1970746,3.000,3.000,8.000,3.000,1312.000,1312.000,...,2016.000,45726.000,1533.890,60371236012000.000,4,0.007,2017-01-01,Central,Condominium,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,266.000,2.000,10833991,2864704,3.000,3.000,8.000,3.000,1741.000,1741.000,...,2016.000,114000.000,4685.340,60371319001006.000,77608,-0.002,2017-09-20,Central,Condominium,nan
77575,261.000,2.000,11000655,673515,2.000,2.000,6.000,2.000,1286.000,1286.000,...,2016.000,283704.000,4478.430,60371014003002.000,77609,0.021,2017-09-20,Central,Single Family Residential,nan
77576,261.000,nan,17239384,2968375,2.000,4.000,nan,2.000,1612.000,1612.000,...,2016.000,16522.000,1107.480,61110084022016.000,77610,0.013,2017-09-21,None,Single Family Residential,nan
77577,261.000,2.000,12773139,1843709,1.000,3.000,4.000,1.000,1032.000,1032.000,...,2016.000,16749.000,876.430,60374338022005.000,77611,0.037,2017-09-21,Central,Single Family Residential,nan


Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.